## QR Decomposition

In [383]:
def Householder_Transform(u, A):
    """
    Computes the Householder transform defined by u of a matrix/vector A
    Note: No need to compute the explicit form of the reflection matrix Q
    
    Input:  u the normal vector defines the reflection plane, nonzero
            A the matrix/vector to be transformed
    
    Output: B the matrix/vector after transformation
    
    Author: Yin Fu 
    Last Edited: 6-30-2022
    """
    import numpy as np
    n = A.shape[0]
    beta = 2/np.dot(u, u)      # Avoiding repetitive computation
    
    # Transform a matrix
    if A.ndim==2:
        B = np.zeros(shape=(n, n))
        for i, v in enumerate(A.T):
            B[:, i] = v - beta*np.dot(v, u)*u
        return B
    
    # Transform a vector
    if A.ndim==1:
        return A - beta*np.dot(A, u)*u

In [385]:
def QR_factorization(A):
    """
    Computes the QR_factorization of a matrix A = QR using Householder transform
    where Q is an orthogonal matrix
          R is an uppertriangular matrix
    
    Input:  A the original matrix
    
    Output: Q the othogonal matrix
            R the uppertriangular matrix
            U the normal vectors that generates Q sequentially 
    
    Author: Yin Fu 
    Last Edited: 6-30-2022
    """
    import numpy as np
    
    # Dimension of the input matrix
    n = A.shape[0]
    
    # Factorization
    U = np.zeros(shape=(n,n-1))       # Storing the normal vector u for each reflection matrix Q
    R = np.zeros(shape=(n,n))
    Q = np.eye(n)
    for i in range(n-1):              # No need to make any adjustment to the last column
        u = np.zeros(n)
        sign = 1 if A[i,i]<0 else -1
        u[i] = A[i,i] - sign*np.linalg.norm(A[i:,i])
        u[i+1:] = A[i+1:,i]
        
        # Reflection vector
        U[:, i] = u
        
        # Update matrix A and Q
        A = Householder_Transform(u, A)
        Q = Householder_Transform(u, Q)
    
    Q = Q.T
    R = A
    return Q, R, U

## LU Decomposition

In [564]:
def LU(A):
    """
    Computes the LU factorization of a square matrix A with partial piviting
    PA = LU
    
    Input:  A the original matrix
    Output: P the permutation matrix
            L the lower triangular matrix
            U the upper triangular matrix
    
    Author: Yin Fu
    Last Edited: 7-2-2022
    """
    import numpy as np
    
    def permute(i, j, A):
        """
        Permutes the ith and jth row for a given square matrix A
        Output: P the permutation matrix 
                A the matrix after the permutation
        """
        n = A.shape[0]
        A_permute = A.copy()
        A_permute[[i, j]] = A_permute[[j, i]]
        
        P = np.eye(n)
        P[[i, j]] = P[[j, i]]
        return P, A_permute
    
    def nonzero_idx(v, i=0):
        """
        Finds the first nonzero index of a given vector v after index i (i not included)
        Output: j the index of the first non-zero entry
        """
        v_partial = v[i+1:]
        for j,val in enumerate(v_partial):
            if val!=0:
                return (i+1)+j
        return "All Zeros"
    
    def reduce(i, A):
        """
        Computes the L_i matrix that performs the Gaussian-Elimination
        for the ith(index) col of matrix A where the leading term of ith(index) col not 0
        """
        L = np.eye(A.shape[0])
        for j in range(i+1, A.shape[0]):
            L[j,i] = -A[j,i]/A[i,i]
        return L
        
    n = A.shape[0]
    A_i = A.copy()
    
    P = np.eye(n)                               # the final P matrix                                                             
    L = np.eye(n)                               # the final L matrix
    U = np.eye(n)                               # the final U matrix
    Lbds = []                                   # the Lambda matrices
    
    for i in range(n-1):
        v = A_i[:,i]
        idx = nonzero_idx(v,i)
        
        if v[i]!=0:
            L_i = reduce(i, A_i)                # Compute the L_i matrix
            P_i = np.eye(n)                     # Compute the P_i matrix
        elif idx =="All Zeros":
            L_i = np.eye(n)
            P_i = np.eye(n)
        else:
            P_i, A_i = permute(i, idx, A_i)
            L_i = reduce(i, A_i)
        P = P_i@P                               # Update the P matrix at each step
        
        Lbds = [(P_i@np.array(L_i)@P_i).tolist() for L_i in Lbds]    # Compute the Lambda matrices
        Lbds.append(L_i.tolist())
        
        A_i = L_i@A_i
    
    for i in range(n-1):                        # Compute the L matrix
        L = L@np.linalg.inv(np.array(Lbds[i]))
        
    U = A_i                                     # Compute the U matrix
    return P, L, U

## Cholesky Decomposition

In [646]:
def Cholesky(A, method='Banachiewicz'):
    """
    Computes the Cholesky decomposition of a positive-definite matrix A.
    A = LL^T where L is lower triangular.
    
    Input:  A an positive-definite matrix
            method the method used to implement the algorithm.
                'Banachiewizc' the Cholesky-Banachiewicz algorithm - positive semidefinite matrices
                'LU' the LU decomposition - positive definite matrix only
    
    Output: L the lower triangular matrix
            L.T the transpose of L
            
    Auther: Yin Fu
    Last Edited: 7-3-2022
    """
    import numpy as np
    
    if method=='LU':
        P, L, U = LU(A)
        D = np.diag(np.diag(U))

        return L@np.sqrt(D)

    elif method=='Banachiewicz':
        n = A.shape[0]
        L = np.eye(n)
        for i in range(n):
            for j in range(i+1):
                
                sum = 0
                for k in range(j):
                    sum += L[i,k]*L[j,k]
                    
                if i==j:
                    L[i,j] = np.sqrt(A[i,j] - sum)
                else:
                    L[i,j] = 1/L[j,j] * (A[i,j] - sum)
        return L